In [0]:
#handled Data Ingestion (Phase 1), Advanced Transformation (Phase 2), and Mathematical Optimization (Phase 3), the final step is to build an End-to-End Pipeline Orchestrator and a Final Results Table
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from pyspark.sql import functions as F

print("--- Starting End-to-End Supply Chain Optimization ---")

# 1. Update/Cache the 'Gold' Inventory Master Table
# This ensures we have the latest stock, ABC categories, and Risk Scores
inventory_gold = spark.table("supply_chain_opt.gold_inventory_master")

# 2. Filter for 'A' items with 'CRITICAL' risk
# We limit to top 10 to ensure the solver remains feasible for this demo
optimization_candidates = inventory_gold \
    .filter("abc_category = 'A' AND priority_level = 'CRITICAL'") \
    .limit(10).toPandas()

if not optimization_candidates.empty:
    print(f"Optimizing {len(optimization_candidates)} critical items...")
    
    # --- PHASE 3: THE TWO-PHASE SIMPLEX SOLVER ---
    
    # Objective: Maximize Risk Reduction (Minimize negative Risk Scores)
    c = optimization_candidates['stock_out_risk_score'].values * -1 
    
    # Inequality Constraint: Total Spend <= Budget
    # We set a generous $100,000 budget to avoid 'Infeasible' status
    budget_limit = 1000000
    A_ub = [optimization_candidates['unit_cost'].values]
    b_ub = [budget_limit] 
    
    # Boundary Constraints: Each order must be at least the 'shortfall'
    # Shortfall = Reorder Point - Current Stock
    bounds = [(row['reorder_point'] - row['current_stock'], None) for _, row in optimization_candidates.iterrows()]
    
    # Execute Solver using the 'highs' method (Standard for Two-Phase)
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    
    if res.success:
        # Attach the optimized quantities to our dataframe
        optimization_candidates['optimized_order_qty'] = np.round(res.x)
        optimization_candidates['total_investment'] = optimization_candidates['optimized_order_qty'] * optimization_candidates['unit_cost']
        
        # 3. Save the Final Procurement Plan back to a Delta Table
        # Use overwriteSchema=True to ensure the new 'optimized' columns are saved
        final_spark_df = spark.createDataFrame(optimization_candidates)
        final_spark_df.write.mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable("supply_chain_opt.final_procurement_plan")
        
        print(f"Success! Optimal plan found. Total Spend: ${optimization_candidates['total_investment'].sum():,.2f}")
    else:
        print(f"Solver Error: {res.message}. Try increasing the budget or reducing item count.")
else:
    print("No critical items found today. Inventory is within stable limits.")

print("--- Pipeline Execution Complete ---")

--- Starting End-to-End Supply Chain Optimization ---
Optimizing 10 critical items...
Success! Optimal plan found. Total Spend: $999,865.50
--- Pipeline Execution Complete ---


In [0]:
%sql
SELECT 
    product_name, 
    abc_category, 
    current_stock, 
    reorder_point, 
    optimized_order_qty,
    round(total_investment, 2) as spend
FROM supply_chain_opt.final_procurement_plan
WHERE optimized_order_qty > 0
ORDER BY spend DESC;

product_name,abc_category,current_stock,reorder_point,optimized_order_qty,spend
Item_284,A,1,123,1558.0,631846.9
Item_135,A,73,199,126.0,55732.32
Item_134,A,44,238,194.0,48150.8
Item_209,A,41,192,151.0,45728.84
Item_270,A,78,216,138.0,44135.16
Item_77,A,107,242,135.0,40641.75
Item_171,A,77,209,132.0,38400.12
Item_208,A,82,167,85.0,38347.75
Item_29,A,50,197,147.0,34777.26
Item_278,A,33,93,60.0,22104.6
